In [1]:
from tensorflow.keras.datasets import cifar10

from ctypes import CDLL, Structure, c_uint, c_float, c_ubyte, POINTER, pointer

import numpy as np
from termcolor import colored

import matplotlib.pyplot as plt
from matplotlib import colors
%matplotlib notebook

In [2]:
# load dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
# conversion and preprocessing
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)
x_train /= 255
x_test /= 255

In [4]:
# custom interface functions
lib = CDLL("./libdeep_cyber_cifar10.so")

def unpackIndex(i, default):
    a = b = c = d = default
    if type(i) == int:
        d = i
    elif len(i) == 1:
        d = i[0]
    elif len(i) == 2:
        c = i[0]
        d = i[1]
    elif len(i) == 3:
        b = i[0]
        c = i[1]
        d = i[2]
    else:
        a = i[0]
        b = i[1]
        c = i[2]
        d = i[3]
    return (a, b, c, d)

class Tensor(Structure):
    _fields_ = [("a", c_uint),
               ("b", c_uint),
               ("c", c_uint),
               ("d", c_uint),
               ("data", POINTER(c_float))]

    def __init__(self, i):
        if type(i) == tuple:
            (a, b, c, d) = unpackIndex(i, 1)
            lib.create_tensor.argtypes = [c_uint, c_uint, c_uint, c_uint]
            lib.create_tensor.restype = Tensor
            t = lib.create_tensor(a, b, c, d)
            self.a = t.a
            self.b = t.b
            self.c = t.c
            self.d = t.d
            self.data = t.data
            t.data = None
        elif type(i) == np.ndarray:
            # create new tensor
            (a, b, c, d) = unpackIndex(i.shape, 1)
            lib.create_tensor.argtypes = [c_uint, c_uint, c_uint, c_uint]
            lib.create_tensor.restype = Tensor
            
            # copy data from numpy array
            t = lib.create_tensor(1, 1, 1, a*b*c*d)
            i = i.flatten()
            for x in range(t.d):
                t[x] = float(i[x])
                
            # move data 
            self.data = t.data
            t.data = None
            self.a = a
            self.b = b
            self.c = c
            self.d = d
        else:
            raise("Illegal input type!")

    def __del__(self):
        lib.free_tensor.argtypes = [Tensor]
        lib.free_tensor(self)

    def __getitem__(self, i):
        (a, b, c, d) = unpackIndex(i, 0)
        lib.at.argtypes = [POINTER(Tensor), c_uint, c_uint, c_uint, c_uint]
        lib.at.restype = POINTER(c_float)
        return lib.at(pointer(self), a, b, c, d)[0]

    def __setitem__(self, i, v):
        (a, b, c, d) = unpackIndex(i, 0)
        lib.at.argtypes = [POINTER(Tensor), c_uint, c_uint, c_uint, c_uint]
        lib.at.restype = POINTER(c_float)
        lib.at(pointer(self), a, b, c, d)[0] = float(v)

    @property
    def shape(self):
        if self.a != 1 and self.b != 1 and self.c != 1 and self.d != 1:
            return (self.a, self.b, self.c, self.d)
        elif self.b != 1 and self.c != 1 and self.d != 1:
            return (self.b, self.c, self.d)
        elif self.c != 1 and self.d != 1:
            return (self.c, self.d)
        else:
            return (self.d, )
        
    def reshape(self, i):
        (self.a, self.b, self.c, self.d) = unpackIndex(i, 1)
        
    def numpy(self):
        t = np.zeros((self.a, self.b, self.c, self.d))
        for ai in range(self.a):
            for bi in range(self.b):
                for ci in range(self.c):
                    for di in range(self.d):
                        t[ai, bi, ci, di] = self[ai, bi, ci, di]
        return t
    
def run_model(X):
    X = Tensor(X)
    lib.cifar10.argtypes = [Tensor]
    lib.cifar10.restype = Tensor
    return lib.cifar10(X)

In [5]:
# execute model on test data
classes = ("airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")

c1 = [np.load("c1w.npy"), np.load("c1b.npy")]
c2 = [np.load("c2w.npy"), np.load("c2b.npy")]
c3 = [np.load("c3w.npy"), np.load("c3b.npy")]
c4 = [np.load("c4w.npy"), np.load("c4b.npy")]
d1 = [np.load("d1w.npy"), np.load("d1b.npy")]
d2 = [np.load("d2w.npy"), np.load("d2b.npy")]

res = run_model(np.array(x_test[0:10])).numpy().flatten()

expected = [int(y_test[i]) for i in range(10)]

for i in range(10):
    #plt.figure(i)
    #plt.imshow(x_test[i])
    #plt.show()
    
    color = "green"
    if int(y_test[i]) != int(res[i]):
        color = "red"
    
    print(i, "-- Reference:", classes[int(y_test[i])], colored("\tResult: " + classes[int(res[i])], color))

0 -- Reference: cat 	Result: dog
1 -- Reference: ship 	Result: ship
2 -- Reference: ship 	Result: ship
3 -- Reference: airplane 	Result: airplane
4 -- Reference: frog 	Result: frog
5 -- Reference: frog 	Result: frog
6 -- Reference: automobile 	Result: automobile
7 -- Reference: frog 	Result: frog
8 -- Reference: cat 	Result: cat
9 -- Reference: automobile 	Result: automobile
